Librerias

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib # Para guardar el scaler

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


Cargar y Explorar los Datos

In [2]:
# Cargar el dataset desde el archivo CSV

df = pd.read_csv('../data/cancer_dataset.csv')

# Vistazo rápido a los datos
print("Primeras 5 filas:")
print(df.head())

print("\nInformación del DataFrame:")
df.info()

print("\nEstadísticas descriptivas:")
print(df.describe())

print("\nValores nulos por columna:")
print(df.isnull().sum())

Primeras 5 filas:
   id  age  gender   bmi alcohol_consumption smoking_status  hepatitis_b  \
0   1   68  Female  18.1             Regular         Former            0   
1   2   81  Female  19.9          Occasional          Never            0   
2   3   58  Female   5.5               Never          Never            0   
3   4   44    Male  16.0               Never         Former            0   
4   5    7    Male   1.0          Occasional         Former            0   

   hepatitis_c  liver_function_score  alpha_fetoprotein_level  \
0            0                  51.9                    16.44   
1            0                  41.6                     8.09   
2            0                  76.0                     0.64   
3            0                  50.3                    19.09   
4            0                  39.5                     4.95   

   cirrhosis_history  family_history_cancer physical_activity_level  diabetes  \
0                  0                      0          

Preprocesamiento de Datos

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer # <--- Importante
from sklearn.pipeline import Pipeline     # <--- Importante

# 1. Separar características (X) y variable objetivo (y)
X = df.drop(['id', 'liver_cancer'], axis=1)
y = df['liver_cancer']

# 2. Identificar columnas numéricas y categóricas
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# 3. Crear pipelines de transformación
# Pipeline para datos numéricos: 
#   Paso 1: Imputar NaNs con la media
#   Paso 2: Escalar los datos
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # Rellenar valores faltantes
    ('scaler', StandardScaler())                # Escalar
])

# Pipeline para datos categóricos:
#   Paso 1: Codificar (One-Hot)
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore')) # Ignorar categorías no vistas en 'test'
])

# 4. Crear el preprocesador que aplica los pipelines a las columnas correctas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# 5. Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 6. Aplicar el preprocesador
# Se ajusta con los datos de entrenamiento y se transforma
X_train_processed = preprocessor.fit_transform(X_train)

# Solo se transforma en los datos de prueba
X_test_processed = preprocessor.transform(X_test)

print("Datos preprocesados y listos (con imputación).")
print("Forma de X_train_processed:", X_train_processed.shape)
print("Forma de X_test_processed:", X_test_processed.shape)

Datos preprocesados y listos (con imputación).
Forma de X_train_processed: (4000, 20)
Forma de X_test_processed: (1000, 20)


Construcción del Modelo de Deep Learning (MLP)

In [4]:
# Construir la arquitectura de la Red Neuronal Multicapa (MLP)
model = Sequential([
    # Capa de entrada: el shape debe coincidir con el número de características
   
    Dense(32, activation='relu', input_shape=(X_train_processed.shape[1],)),
    
    Dropout(0.2), # Capa de regularización para evitar sobreajuste
    Dense(16, activation='relu'),
    
    # Capa de salida: 1 neurona con activación sigmoide para predicción binaria (probabilidad)
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Proyecto_IA\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,217 (4.75 KB)

 Trainable params: 1,217 (4.75 KB)

 Non-trainable params: 0 (0.00 B)

Entrenar el Modelo

In [5]:
# Entrenar el modelo con los datos de entrenamiento

history = model.fit(X_train_processed, y_train,
                    epochs=50, # Número de veces que el modelo verá todos los datos
                    batch_size=32,
                    validation_split=0.1, # Usar una parte de los datos de entrenamiento para validar en cada época
                    verbose=1)

Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7842 - loss: 0.4793 - val_accuracy: 0.8375 - val_loss: 0.3675
Epoch 2/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8433 - loss: 0.3619 - val_accuracy: 0.8775 - val_loss: 0.2925
Epoch 3/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8636 - loss: 0.3164 - val_accuracy: 0.8750 - val_loss: 0.2684
Epoch 4/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8686 - loss: 0.3042 - val_accuracy: 0.8825 - val_loss: 0.2600
Epoch 5/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8753 - loss: 0.2887 - val_accuracy: 0.8775 - val_loss: 0.2531
Epoch 6/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8808 - loss: 0.2836 - val_accuracy: 0.8775 - val_loss: 0.2510
Epoch 7/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8758 - loss: 0.2812 - val_accuracy: 0.8800 - val_loss: 0.2490
Epoch 8/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8847 - loss: 0.2750 - val_accuracy: 0.

Evaluar el Modelo

In [6]:
# Evaluar el rendimiento en el conjunto de prueba

loss, accuracy = model.evaluate(X_test_processed, y_test)
print(f"\nAccuracy en el conjunto de prueba: {accuracy*100:.2f}%")

# Calcular el AUC (Área Bajo la Curva ROC), una métrica robusta

y_pred_proba = model.predict(X_test_processed).flatten()
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC en el conjunto de prueba: {auc:.4f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8980 - loss: 0.2296 

Accuracy en el conjunto de prueba: 89.80%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
AUC en el conjunto de prueba: 0.9444


Guardar el Modelo y el Scaler

In [7]:
# Guardar el modelo entrenado
model.save('modelo_cancer.h5')
print("Modelo guardado como 'modelo_cancer.h5'")

# Guardar el objeto PREPROCESSOR completo
# Este es el archivo que usará el back-end
joblib.dump(preprocessor, 'preprocessor.pkl') 
print("¡Correcto! Preprocessor guardado como 'preprocessor.pkl'")

Modelo guardado como 'modelo_cancer.h5'
¡Correcto! Preprocessor guardado como 'preprocessor.pkl'
